In [4]:
# -*- coding: utf-8 -*-
"""
YOLOv8 Dataset Preparation and Evaluation Script
Includes: Training, Testing, Metrics Export, Graphs, and Sample Predictions
"""

import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO

# --------------------------------------------------
# 1. Define Paths and Parameters
# --------------------------------------------------

# Input directories
orig_img_dir    = Path(r"C:\Users\alexi\Downloads\training_yolo_1_source\NAC")
orig_label_dir  = Path(r"C:\Users\alexi\Downloads\training_yolo_1_source\nac_bb")
gen_img_dir     = Path(r"C:\Users\alexi\Downloads\generated")

# Output directories
proc_label_dir  = Path("processed_labels")
dataset_dir     = Path("dataset_yolo")
yolo_out_dir    = Path("Yolo")
yolo_out_dir.mkdir(exist_ok=True)

# Dataset split ratios
test_ratio_nac = 0.1
test_ratio_gen = 0.1
train_ratio    = 0.8
val_ratio      = 0.1

# Image size settings
original_size = (199, 199)
target_size   = (256, 256)

# --------------------------------------------------
# 2. Create Directories for Processed Labels
# --------------------------------------------------
for subset in ["nac", "gen"]:
    (proc_label_dir / subset).mkdir(parents=True, exist_ok=True)

# --------------------------------------------------
# 3. Convert Bounding Boxes to YOLO Format
# --------------------------------------------------
def convert_and_save_label(src_path, dst_path, orig_size, target_size):
    with src_path.open('r') as f:
        lines = f.read().strip().splitlines()
    if not lines:
        return
    ow, oh = orig_size
    tw, th = target_size
    sx, sy = tw / ow, th / oh
    with dst_path.open('w') as f:
        for line in lines:
            x1, y1, x2, y2 = map(float, line.split())
            x1_s, y1_s = x1 * sx, y1 * sy
            x2_s, y2_s = x2 * sx, y2 * sy
            xc = (x1_s + x2_s) / 2.0
            yc = (y1_s + y2_s) / 2.0
            w  = x2_s - x1_s
            h  = y2_s - y1_s
            f.write(f"0 {xc/tw:.6f} {yc/th:.6f} {w/tw:.6f} {h/th:.6f}\n")

# --------------------------------------------------
# 4. Process Labels for NAC and Generated Images
# --------------------------------------------------
for img_path in orig_img_dir.glob("*.png"):
    src = orig_label_dir / f"{img_path.stem}.txt"
    dst = proc_label_dir / "nac" / f"{img_path.stem}.txt"
    if src.exists():
        convert_and_save_label(src, dst, original_size, target_size)

for img_path in gen_img_dir.glob("*_GEN.png"):
    base = img_path.stem.replace("_GEN", "")
    src = orig_label_dir / f"{base}.txt"
    dst = proc_label_dir / "gen" / f"{img_path.stem}.txt"
    if src.exists():
        convert_and_save_label(src, dst, original_size, target_size)

# --------------------------------------------------
# 5. Match Image-Label Pairs
# --------------------------------------------------
nac_pairs = [(i, proc_label_dir / "nac" / f"{i.stem}.txt") for i in orig_img_dir.glob("*.png") if (proc_label_dir / "nac" / f"{i.stem}.txt").exists()]
gen_pairs = [(i, proc_label_dir / "gen" / f"{i.stem}.txt") for i in gen_img_dir.glob("*_GEN.png") if (proc_label_dir / "gen" / f"{i.stem}.txt").exists()]

# --------------------------------------------------
# 6. Split Dataset into Train, Validation, Test
# --------------------------------------------------
nac_train_val, nac_test = train_test_split(nac_pairs, test_size=test_ratio_nac, random_state=42)
gen_train_val, gen_test = train_test_split(gen_pairs, test_size=test_ratio_gen, random_state=42)
train_val = nac_train_val + gen_train_val
val_fraction = val_ratio / (train_ratio + val_ratio)
train_pairs, val_pairs = train_test_split(train_val, test_size=val_fraction, random_state=42)

# --------------------------------------------------
# 7. Create YOLO-Compatible Directory Structure
# --------------------------------------------------
for split in ["train", "val"]:
    (dataset_dir / "images" / split).mkdir(parents=True, exist_ok=True)
    (dataset_dir / "labels" / split).mkdir(parents=True, exist_ok=True)
for kind in ["nac", "gen"]:
    (dataset_dir / "images" / "test" / kind).mkdir(parents=True, exist_ok=True)
    (dataset_dir / "labels" / "test" / kind).mkdir(parents=True, exist_ok=True)

# --------------------------------------------------
# 8. Copy Images and Labels to the Final Structure
# --------------------------------------------------
def copy_set(pairs, split):
    for img, lbl in pairs:
        if split in ["train", "val"]:
            shutil.copy(img, dataset_dir / "images" / split / img.name)
            shutil.copy(lbl, dataset_dir / "labels" / split / lbl.name)
        else:
            kind = split.split("/")[1]
            shutil.copy(img, dataset_dir / "images" / "test" / kind / img.name)
            shutil.copy(lbl, dataset_dir / "labels" / "test" / kind / lbl.name)

copy_set(train_pairs, "train")
copy_set(val_pairs, "val")
copy_set(nac_test, "test/nac")
copy_set(gen_test, "test/gen")

# --------------------------------------------------
# 9. Generate dataset.yaml File
# --------------------------------------------------
with open(dataset_dir / 'dataset.yaml', 'w') as f:
    yaml.dump({'path': str(dataset_dir), 'train': 'images/train', 'val': 'images/val', 'nc': 1, 'names': ['nac']}, f)

# --------------------------------------------------
# 10. Train YOLOv8n Model
# --------------------------------------------------
model = YOLO('yolov8n.pt')
model.train(data=str(dataset_dir / 'dataset.yaml'), epochs=50, imgsz=256, project='runs/train', name='exp1')

# --------------------------------------------------
# 11. Evaluate Model on NAC and Generated Test Sets
# --------------------------------------------------
results = []
for name in ["nac", "gen"]:
    test_yaml = dataset_dir / f'{name}_test.yaml'
    with open(test_yaml, 'w') as f:
        yaml.dump({'path': str(dataset_dir), 'train': 'images/train', 'val': f'images/test/{name}', 'nc': 1, 'names': ['nac']}, f)
    metrics = model.val(data=str(test_yaml), imgsz=256, save_json=True)
    row = {
        'Dataset': name.upper(),
        'mAP@50': metrics.box.map50,
        'mAP@50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr
    }
    results.append(row)

# Save metrics to CSV
df = pd.DataFrame(results)
df.to_csv(yolo_out_dir / "metrics.csv", index=False)

# --------------------------------------------------
# 12. Plot and Save Evaluation Graphs
# --------------------------------------------------
df.set_index("Dataset")[["mAP@50", "mAP@50-95"]].plot(kind='bar', title='mAP Comparison', figsize=(10,5))
plt.ylabel("Score")
plt.ylim(0, 1.05)
plt.grid(True)
plt.savefig(yolo_out_dir / "map_comparison.png")
plt.close()

df.set_index("Dataset")[["Precision", "Recall"]].plot(kind='bar', title='Precision & Recall Comparison', figsize=(10,5))
plt.ylabel("Score")
plt.ylim(0, 1.05)
plt.grid(True)
plt.savefig(yolo_out_dir / "pr_comparison.png")
plt.close()

# --------------------------------------------------
# 13. Save Example Predictions
# --------------------------------------------------
nac_dir = dataset_dir / "images/test/nac"
gen_dir = dataset_dir / "images/test/gen"
samples = []
for img_path in sorted(nac_dir.glob("*.png")):
    gen_path = gen_dir / f"{img_path.stem}_GEN.png"
    if gen_path.exists():
        samples.append((img_path, gen_path))
    if len(samples) == 5:
        break

def save_prediction(img_path, save_path):
    r = model(str(img_path))[0]
    img = cv2.imread(str(img_path))
    for box in r.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = map(int, box[:4])
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.imwrite(str(save_path), img)

for nac_img, gen_img in samples:
    save_prediction(nac_img, yolo_out_dir / f"pred_{nac_img.name}")
    save_prediction(gen_img, yolo_out_dir / f"pred_{gen_img.name}")

print("✅ All steps completed. Results saved in 'Yolo' folder.")


Ultralytics 8.3.153  Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_yolo\dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp112, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

train: Scanning C:\Users\alexi\dataset_yolo\labels\train.cache... 942 images, 0 backgrounds, 0 corrupt: 100%|██████████

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access  (ping: 0.10.0 ms, read: 3.54.0 MB/s, size: 3.3 KB)


val: Scanning C:\Users\alexi\dataset_yolo\labels\val.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11


...

                   all        118        118      0.964      0.914      0.965      0.676
Speed: 0.4ms preprocess, 1.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\train\exp112
Ultralytics 8.3.153  Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 1.61.7 MB/s, size: 5.1 KB)


val: Scanning C:\Users\alexi\dataset_yolo\labels\test\nac.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<0


                   all         59         59      0.914        0.9      0.932      0.622
Speed: 0.7ms preprocess, 4.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Saving runs\train\exp1122\predictions.json...
Results saved to runs\train\exp1122
Ultralytics 8.3.153  Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 1.41.1 MB/s, size: 2.7 KB)


val: Scanning C:\Users\alexi\dataset_yolo\labels\test\gen.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<0


                   all         59         59      0.963      0.966      0.991      0.747
Speed: 0.9ms preprocess, 4.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Saving runs\train\exp1123\predictions.json...
Results saved to runs\train\exp1123

image 1/1 C:\Users\alexi\dataset_yolo\images\test\nac\img_164.png: 256x256 1 nac, 10.4ms
Speed: 1.6ms preprocess, 10.4ms inference, 2.5ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 C:\Users\alexi\dataset_yolo\images\test\gen\img_164_GEN.png: 256x256 1 nac, 10.4ms
Speed: 1.2ms preprocess, 10.4ms inference, 2.2ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 C:\Users\alexi\dataset_yolo\images\test\nac\img_326.png: 256x256 1 nac, 10.4ms
Speed: 2.9ms preprocess, 10.4ms inference, 2.4ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 C:\Users\alexi\dataset_yolo\images\test\gen\img_326_GEN.png: 256x256 1 nac, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.3ms postprocess per image at shape (1, 3, 2